In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Optional: make plots look nicer
sns.set(style="whitegrid", context="notebook")

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


In [2]:
data= pd.read_csv('train.csv')
data.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


a)Look for the missing values in all the columns and either impute them (replace with mean,
median, or mode) or drop them. Justify your action for this task. (4 points)





In [3]:
data.info()

print("\nMissing values per column:")
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB

Missing values per column:


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


imputing missing values

In [4]:
cars_imputed = data.copy()

# Identify numeric and categorical columns
numeric_cols = cars_imputed.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = cars_imputed.select_dtypes(exclude=[np.number]).columns.tolist()

numeric_cols, categorical_cols


(['Unnamed: 0', 'Year', 'Kilometers_Driven', 'Seats', 'Price'],
 ['Name',
  'Location',
  'Fuel_Type',
  'Transmission',
  'Owner_Type',
  'Mileage',
  'Engine',
  'Power',
  'New_Price'])

b)Remove the units from some of the attributes and only keep the numerical values (for
example remove kmpl from “Mileage”, CC from “Engine”, bhp from “Power”, and lakh from
“New_price”). (4 points)

In [5]:
cars_clean = cars_imputed.copy()

def extract_number(s):
    if pd.isna(s):
        return np.nan
    # keep digits, decimal point and possibly minus sign
    import re
    match = re.findall(r"[-+]?\d*\.?\d+", str(s))
    if match:
        return float(match[0])
    return np.nan

for col in ["Mileage", "Engine", "Power", "New_price"]:
    if col in cars_clean.columns:
        cars_clean[col + "_num"] = cars_clean[col].apply(extract_number)

cars_clean[[c for c in cars_clean.columns if "Mileage" in c or
                                      "Engine" in c or
                                      "Power" in c or
                                      "New_price" in c]].head()


,Mileage,Engine,Power,Mileage_num,Engine_num,Power_num
0,19.67 kmpl,1582 CC,126.2 bhp,19.67,1582.0,126.20
1,13 km/kg,1199 CC,88.7 bhp,13.00,1199.0,88.70
2,20.77 kmpl,1248 CC,88.76 bhp,20.77,1248.0,88.76
3,15.2 kmpl,1968 CC,140.8 bhp,15.20,1968.0,140.80
4,23.08 kmpl,1461 CC,63.1 bhp,23.08,1461.0,63.10


 c)Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot
encoded value. (4 points).

In [6]:
cols_to_encode = [c for c in ["Fuel_Type", "Transmission"] if c in cars_clean.columns]

cars_encoded = pd.get_dummies(cars_clean, columns=cols_to_encode, drop_first=True)
cars_encoded.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Mileage_num,Engine_num,Power_num,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,19.67,1582.0,126.20,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,13.00,1199.0,88.70,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,20.77,1248.0,88.76,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,15.20,1968.0,140.80,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50,23.08,1461.0,63.10,False,False,True


d)Create one more feature and add this column to the dataset (you can use mutate function in
R for this). For example, you can calculate the current age of the car by subtracting “Year” value
from the current year. (4 points)

In [7]:
CURRENT_YEAR = 2025

if "Year" in cars_encoded.columns:
    cars_encoded["Car_Age"] = CURRENT_YEAR - cars_encoded["Year"]

cars_encoded[["Year", "Car_Age"]].head()

,Year,Car_Age
0,2015,10
1,2011,14
2,2012,13
3,2013,12
4,2013,12


In [8]:
df = cars_encoded.copy()

# SELECT: choose some columns
selected = df[["Make", "Model", "Price", "Car_Age"]] if set(["Make","Model","Price","Car_Age"]).issubset(df.columns) else df.head()
selected.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Mileage_num,Engine_num,Power_num,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,19.67,1582.0,126.20,False,False,True,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,13.00,1199.0,88.70,False,True,True,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,20.77,1248.0,88.76,False,False,True,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,15.20,1968.0,140.80,False,False,False,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50,23.08,1461.0,63.10,False,False,True,12


 e)Perform select, filter, rename, mutate, arrange and summarize with group by operations (or
their equivalent operations in python) on this dataset. (4 points)


In [9]:
# FILTER: cars with price > 5 lakh and petrol fuel (if column exists)
if "Price" in df.columns:
    filtered = df[df["Price"] > 5]

    # If we have dummy column e.g. Fuel_Type_Petrol (depends on your dummies)
    petrol_cols = [c for c in df.columns if "Fuel" in c and "Petrol" in c]
    if petrol_cols:
        filtered = filtered[filtered[petrol_cols[0]] == 1]

    filtered.head()


In [10]:
# RENAME: rename columns
renamed = df.rename(columns={"Price": "Resale_Price"})
renamed.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Resale_Price,Mileage_num,Engine_num,Power_num,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50,19.67,1582.0,126.20,False,False,True,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50,13.00,1199.0,88.70,False,True,True,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00,20.77,1248.0,88.76,False,False,True,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74,15.20,1968.0,140.80,False,False,False,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50,23.08,1461.0,63.10,False,False,True,12


In [11]:
# ARRANGE: sort by price descending
if "Price" in df.columns:
    arranged = df.sort_values(by="Price", ascending=False)
    arranged.head()


In [12]:
# SUMMARIZE with GROUP BY: average price by Fuel_Type (if original column exists)
if "Fuel_Type" in cars_clean.columns and "Price" in cars_clean.columns:
    summary = cars_clean.groupby("Fuel_Type")["Price"].agg(["mean", "median", "count"]).reset_index()
    summary
